## Intro to web scraping with Python

Today, we'll be learning about using Python to collect data publicly available on the Internet through what is commonly referred to as web scraping. 

#### Website 

We'll be scraping a [listing of public financial disclosures](https://extapps2.oge.gov/201/Presiden.nsf/PAS%20Filings%20by%20Date?OpenView) filed by top federal government employees posted by the Office of Government Ethics. This listing is not provided in a downloadable format but instead as an HTML table that we'll be using some common Python packages to extract. 

Before we get started, we'll want to use the inspector in our browser to examine the pattern of how the data is stored and displayed. 

#### Import statements 

To use a Python library in a script, we have to import it. You do that using what are known as import statements. Below we'll import the packages that we installed and that we'll be using to scrape the website.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import pprint

#### Assign the url to a variable

In Python, variables are assigned using the `=` with the variable name to the left and the contents of the variable to the right.

In [ ]:
url = 'https://extapps2.oge.gov/201/Presiden.nsf/PAS%20Filings%20by%20Date?OpenView'

#### Set up our header (Scraping ethically and openly)

When scraping, it is best practice to be transparent about what you're doing and kind to the websites you're visiting. Below we'll construct a header that we'll send along with our http request when we visit the website so that the webmaster of the site would be able to see what we're doing in their logs. In most cases, you just need to include your email address so they know how to contact you. 

In [ ]:
header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36 (chad.day@wsj.com)'}



#### Visit our website (Send an HTTP request with `requests`)

Here we'll use the Python package `requests`, which does basically what your browser does when it visits a website. What we get back is generally referred to as the response. A part of that response is the page HTML. 


In [ ]:
r = requests.get(url, headers=header)

html = r.text

#pprint.pprint(html)

#### Let's look closely at one entry

It's easier to do this using your browser inspector but I've included one of our rows of data below. What patterns do you see? 

#### Parsing HTML 

To collect this data, we'll need to use a process called `parsing` and use a package called `BeautifulSoup`, which allows us to navigate the structure of HTML and leverage the patterns we see to select the pieces and parts we want. 

To do this, we feed in the html text and have BeautifulSoup parse it. We then assign that parsed html the variable name `soup`.

In [ ]:
soup = BeautifulSoup(html, "lxml")

Now, looking closely at our table on the website, we'll want to look for a couple things. 

1. The HTML heirarchy that contains our whole table.
2. The HTML tag for a row
3. The HTML tag for a cell of a row


#### Find the table

Since we're dealing with an HTML table, we'll see that the whole table of data is contained between tags simple called `<table>` and `</ table>`. In BeautifulSoup, we can search across the parsed HTML and look for all instances of a tag using `find_all`.

In [ ]:
table = soup.find_all('table')

table

#### Cutting into `iterable` chunks
Now within the table, we'll want to split out each row so we can do what's called `iterating` through the data. One of the most common ways of iterating in Python is to use what's called a `for loop`. This translates basically to do something for each item in a group. In Python, we'll be creating a list containing each row so we can iterate through each one. 

In [ ]:
rows = table[0].find_all('tr')

rows[0]

Now that we have a list of our rows, let's focus on gathering five pieces of information from each row. 

 - Date
 - Link to filings
 - Name of filing
 - Filer's name
 - Filer's title

These pieces of data are contained in the `<td>` tags, so we'll use them when parsing each row.

#### Creating Python containers for our data

We've located our data. Now we need to set up a mechanism for storing it in Python so we can work with it. We'll use two types of Python data structures for this. One is a list, which we used before. The other is a `dictionary`, which uses keys and values to store data. You can see the difference below. 


```python

## Lists are contained in brackets
a = [1, 2, 3, 4]

## Empty list
a = []

## Dictionary uses brackets and key and value pairs

a = {
    'First_Name' : 'Chad',
    'Last_Name' : 'Day'
}

```

We'll be using a list of dictionaries to store the data we scrape as below.

Let's start by just gathering the first item in each row.

In [ ]:
## Here's our empty list where we'll store each dictionary 
data = []

for row in rows: ## Here's our for loop; note the colon
    tds = row.find_all('td') ## Creates a list of the data in tds
    row_dict = { ## Note we create our dictionary here
        'date': tds[0].text
    }
    data.append(row_dict) ## We use the append method to add to our data list.

print(data[0:5])





## Add the filing name, the filer's name and title

Note that we're using a process called `indexing` here. Computers count from zero so the first item in our list of `tds` is `0`. In Python, we use brackets next to a list to select just that item. 
```
tds[0] ## First item 
tds[2] ## Third item
tds[3] ## Fourth item
```

Below we'll gather the filing name, filer's name and title.

In [ ]:
data = []

for row in rows: ## Here's our for loop; note the colon
    tds = row.find_all('td')
    row_dict = {
        'date': tds[0].text,
        'filer_name': tds[2].text,
        'agency': tds[3].text,
        'title': tds[4].text
    }
    data.append(row_dict)

print(data)

## Extracting urls 

The only thing missing now is the url of the filing. In HTML, it is stored using the `a` tag and specified as `href`. Return to the Inspector on the page or see below.

```
<td nowrap><font size="2" ''face="Verdana">
        <a 
            href= 'https://extapps2.oge.gov/201/Presiden.nsf/PAS+Index/CA12044E3E1DB208852587A40033D508/$FILE/Young, "Shalanda  final278.pdf'> Nominee 278 (12/03/2021)
        </a>
</font></td>

```

We'll use BeautifulSoup here to grab it from the second item in our `tds` list for each row, which as you remember is accessed using `[1]`.

In [ ]:
data = []

for row in rows: ## Here's our for loop; note the colon
    tds = row.find_all('td')
    url = tds[1].find('a').get('href')
    file_name = tds[1].find('a').text
    row_dict = {
        'date': tds[0].text,
        'url': url,
        'file_name': file_name,
        'filing': tds[2].text,
        'filers_name': tds[3].text,
        'title': tds[4].text
    }
    data.append(row_dict)

print(data[0])

Not too bad but we're missing one last thing. The domain of the url is missing. So let's use a process called string concatenation to add it.

In [ ]:
data = []

for row in rows: ## Here's our for loop; note the colon
    tds = row.find_all('td')
    url_start = 'https://extapps2.oge.gov'
    url_end = tds[1].find('a').get('href')
    url = url_start + url_end
    file_name = tds[1].find('a').text
    row_dict = {
        'date': tds[0].text,
        'url': url,
        'file_name': file_name,
        'filer_name': tds[2].text,
        'agency': tds[3].text,
        'title': tds[4].text
    }
    data.append(row_dict)

print(data[0])

In [ ]:
data

Now that we have our data in a list of dictionaries, let's save it to a file. Python has a built in module called `csv` that writes data to a CSV file but I prefer using the data science libary `pandas`, which we imported earlier in the notebook.

`pandas` uses spreadsheet like objects called dataframes that have columns and rows. Now that we have a list of dictionaries containing our data, we can save it as a dataframe. 

In [ ]:
df = pd.DataFrame(data)

df

And save it as a csv.

In [ ]:
df.to_csv('./data.csv', index=False)

Congrats! You have scraped data from a website and saved it to a CSV!

Hopefully, this helped you feel more comfortable working with data in Python. You can also run this as a script as laid out below. 

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import pprint

def main():
    url = 'https://extapps2.oge.gov/201/Presiden.nsf/PAS%20Filings%20by%20Date?OpenView'
    header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36 (my-email-here)'}
    r = requests.get(url, headers=header)
    html = r.text
    soup = BeautifulSoup(html, "lxml")
    table = soup.find_all('table')
    rows = table[0].find_all('tr')
    data = []
    for row in rows: 
        parse_rows(row, data)
    df = pd.DataFrame(data)
    df.to_csv('./data.csv')

def parse_rows(row, data):
    tds = row.find_all('td')
    url_start = 'https://extapps2.oge.gov'
    url_end = tds[1].find('a').get('href')
    url = url_start + url_end
    row_dict = {
        'date': tds[0].text,
        'url': url,
        'filing': tds[2].text,
        'filers_name': tds[3].text,
        'title': tds[4].text
    }
    data.append(row_dict)


main()
    